In [12]:
!pip install gensim==4.2.0

  Using cached gensim-4.2.0-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (24.1 MB)
  Attempting uninstall: gensim
    Found existing installation: gensim 3.4.0
    Uninstalling gensim-3.4.0:
      Successfully uninstalled gensim-3.4.0


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
from tqdm import tqdm
import torch.nn.functional as F
from gensim.models import Word2Vec
from sklearn import metrics
from sklearn.metrics import classification_report
import itertools
import random

/home/eunchong/anaconda3/envs/network_x/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
import torch
import torch.nn as nn

In [3]:
device = 'cuda'

# 313 881


In [4]:
pos_weight = 881/313

In [5]:
class BertModel(nn.Module):
    def __init__(self, voc_size:int=30000, seq_len: int=512, d_model: int=384, d_ff:int=3072, pad_idx: int=1,
                num_encoder: int=12, num_heads: int=12, n_layers=6, dropout: float=0.1):
        super(BertModel, self).__init__()
        self.pad_idx = pad_idx
        self.layers = nn.ModuleList([nn.TransformerEncoderLayer(d_model=d_model, nhead=8, batch_first=True) for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ1 = nn.Tanh()
        self.classifier = nn.Linear(d_model, 1)
        
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        output = input
        for layer in self.layers:
            output = layer(output)
        #output = output.mean(dim=1, keepdim=True)
        output = output[:,-1]
        h_pooled = self.activ1(self.fc(output)) # [batch_size, d_model]
        logits_clsf = self.classifier(h_pooled) # [batch_size, 2]
        return logits_clsf # [B, S, D_model]

In [6]:
def log_metrics(preds, labels):
    preds = torch.stack(preds)
    preds = preds.cpu().detach().numpy()
    labels = torch.stack(labels)
    labels = labels.cpu().detach().numpy()
    
    
    ## Method 2 using ravel()
    fpr_micro, tpr_micro, _ = metrics.roc_curve(labels.ravel(), preds.ravel())
    auc_micro = metrics.auc(fpr_micro, tpr_micro)
    n=10
    preds = [round(i) for i in preds.ravel()]
    acc = metrics.accuracy_score(labels.ravel(), preds)
    f1 = metrics.f1_score(labels.ravel(), preds)
    preds = [preds[i*n:(i+1)*n] for i in range(len(preds)//6)]
    #target_names = list(genre_list.keys())
    #print(classification_report(labels,preds, target_names=target_names))
    return {"auc_micro": auc_micro, "acc" : acc, "f1" : f1, 'pred' : preds}

In [7]:
def train_one_epoch(dataloader, model, optimizer, device, loss_fn):
    model.train()
    tk0 = tqdm(dataloader, total=len(dataloader))
    total_loss = 0.0
    
    for source_tensor, label in tk0:
        optimizer.zero_grad()
        output = model(source_tensor.to(device))
        loss = loss_fn(output.view(-1, output.size(-1)), torch.FloatTensor([label]).to(device))
        
        total_loss += loss
        loss.backward()
        optimizer.step()
        
    avg_train_loss = total_loss / len(dataloader) 
    print(" Average training loss: {0:.2f}".format(avg_train_loss))  

def test_one_epoch(testset, model, loss_fn):
    model.eval()
    tk0 = tqdm(testset, total=len(testset))
    targets = []
    outputs = []
    total_loss = 0.0
    with torch.no_grad():
        for source_tensor, label in tk0:
            label = torch.FloatTensor([label])
            output = model(source_tensor.to(device))
            loss = loss_fn(output.view(-1, output.size(-1)), label.to(device))
            total_loss += loss
            targets.extend(label)
            outputs.extend(torch.sigmoid(output[0]).to('cpu'))
    avg_valid_loss = total_loss / len(testset) 
    score = log_metrics(outputs, targets)
    print(" Average valid loss: {0:.2f}".format(avg_valid_loss))  
    print('AUC_SCORE: ', score["auc_micro"], " acc: ", score["acc"], "f1: ", score["f1"])
    return score, avg_valid_loss
    
def fit(model, train_dataloader, valid_dataloader=None,test_dataloader=None, EPOCHS=3, lr=0.0000005):
    loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weight)) #ignore padding
    optimizer = torch.optim.AdamW(model.parameters(),lr=lr)
    for i in range(EPOCHS):
        print(f"EPOCHS:{i+1}")
        print('TRAIN')
        train_one_epoch(train_dataloader, model, optimizer, device, loss_fn)
        auc, loss = test_one_epoch(valid_dataloader,model, loss_fn)
        if i == 0:
            torch.save(model,'rnn_best.model')
            best_f1 = auc['f1']
        if  best_f1<auc['f1']:
            best_f1 = auc['f1']
            print(i,' EPOCH BEST MODEL!')
            torch.save(model,'rnn_best.model')
        print('Test')
        auc, loss = test_one_epoch(test_dataloader,model, loss_fn)
    best_model = torch.load('rnn_best.model')
    auc, loss  = test_one_epoch(test_dataloader,best_model, loss_fn)
    print('BEST MODEL')
    print('AUC_SCORE: ', auc["auc_micro"], " acc: ", auc["acc"], "f1: ", auc["f1"])
    return auc

            best_f1 = auc['f1']
        if  auc['f1']>best_f1:
            best_f1 = auc['f1']

In [8]:
import pickle
with open('movie_split_cult.pickle','rb') as fw:
    movie_split = pickle.load(fw)

In [9]:
path1 = 'word2vec_model_edge/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
end = torch.ones(1,1,400)
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'cult film'==keyword:
                    cult = 1
                    
                    
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,400)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])

/tmp/ipykernel_2198767/1754640764.py:40: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484810403/work/torch/csrc/utils/tensor_numpy.cpp:172.)
  emb = torch.from_numpy(emb)


In [10]:
bert = BertModel(d_model=400).to('cuda')
score = fit(bert, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:09<00:00, 104.58it/s]


 Average training loss: 1.11


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 728.11it/s]

 Average valid loss: 1.09
AUC_SCORE:  0.5535659154059895  acc:  0.3416666666666667 f1:  0.5093167701863354


...

 Average valid loss: 0.99
AUC_SCORE:  0.7378820623649275  acc:  0.55 f1:  0.578125
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 853.48it/s]


 Average valid loss: 1.03
AUC_SCORE:  0.7240853658536586  acc:  0.5867768595041323 f1:  0.5901639344262294


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 852.66it/s]

 Average valid loss: 1.00
AUC_SCORE:  0.7304878048780488  acc:  0.5867768595041323 f1:  0.5901639344262294
BEST MODEL
AUC_SCORE:  0.7304878048780488  acc:  0.5867768595041323 f1:  0.5901639344262294
{'auc_micro': 0.7304878048780488, 'acc': 0.5867768595041323, 'f1': 0.5901639344262294, 'pred': [[1, 1, 0, 1, 0, 1, 1, 1, 0, 1], [0, 1, 1, 1, 1, 0, 0, 1, 1, 1], [1, 1, 1, 1, 0, 0, 1, 1, 1, 0], [1, 0, 0, 0, 1, 1, 0, 0, 1, 1], [1, 1, 1, 0, 1, 1, 0, 1, 0, 1], [1, 1, 1, 0, 0, 1, 1, 1, 1, 1], [1, 1, 0, 1, 1, 0, 0, 1, 1, 1], [1, 0, 1, 1, 1, 0, 0, 1, 0, 1], [0, 1, 1, 1, 1, 1, 1, 1, 0, 1], [1, 0, 0, 1, 1, 1, 1, 0, 1, 0], [0, 1, 0, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 0, 0, 1, 0, 1, 0, 0], [1], [], [], [], [], [], [], []]}


In [97]:
random.randint(0,100)

44

In [11]:
import random

# SUM

In [12]:
import pickle
with open('emb_sum.pickle','rb') as fw:
    movie_emb = pickle.load(fw)

In [13]:
path1 = 'word2vec_model_char/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
end = torch.ones(1,1,384)

for filename in filenames:
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'cult film'==keyword:
                    cult = 1
    if movie_name not in movie_emb.keys():
        continue
        
    embs = torch.zeros(1,len(movie_emb[movie_name])+1,384)
    for i,emb in enumerate(movie_emb[movie_name]):
        emb = torch.from_numpy(np.nan_to_num(emb))
        embs[0][i] = emb
        
    embs[0][-1] = end
        
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
        #valid_data.append([embs, [cult]])

In [14]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

953
120
119


In [15]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 953/953 [00:07<00:00, 120.16it/s]


 Average training loss: 1.11


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 887.78it/s]


 Average valid loss: 1.09
AUC_SCORE:  0.6137696820006174  acc:  0.3416666666666667 f1:  0.5093167701863354
Test


...

 Average valid loss: 1.04
AUC_SCORE:  0.6810744056807656  acc:  0.5666666666666667 f1:  0.5737704918032787
Test


100%|████████████████████████████████████████| 119/119 [00:00<00:00, 850.96it/s]


 Average valid loss: 0.96
AUC_SCORE:  0.7392120075046904  acc:  0.5882352941176471 f1:  0.5882352941176471


100%|████████████████████████████████████████| 119/119 [00:00<00:00, 853.91it/s]


 Average valid loss: 0.96
AUC_SCORE:  0.7407754846779238  acc:  0.5714285714285714 f1:  0.5853658536585366
BEST MODEL
AUC_SCORE:  0.7407754846779238  acc:  0.5714285714285714 f1:  0.5853658536585366
{'auc_micro': 0.7407754846779238, 'acc': 0.5714285714285714, 'f1': 0.5853658536585366, 'pred': [[1, 1, 0, 1, 1, 1, 1, 1, 0, 1], [0, 1, 1, 1, 1, 0, 0, 1, 1, 1], [1, 1, 1, 1, 0, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 1, 1, 0, 1, 1], [1, 0, 1, 1, 1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 0, 1, 0, 1, 1, 1], [1, 1, 0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 1, 1, 0, 1, 1, 1, 1, 0], [0, 0, 0, 1, 1, 0, 1, 1, 0, 1], [1, 0, 1, 1, 0, 1, 0, 1, 1], [], [], [], [], [], [], []]}


# Doc2vec

In [16]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [17]:
path1 = 'doc2vec/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'cult film'==keyword:
                    cult = 1
        
    w2v=Doc2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.dv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.dv.index_to_key:
            emb = w2v.dv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [18]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:08<00:00, 119.77it/s]


 Average training loss: 1.10


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 939.51it/s]


 Average valid loss: 1.09
AUC_SCORE:  0.610991046619327  acc:  0.3416666666666667 f1:  0.5093167701863354
Test


...

 Average valid loss: 1.27
AUC_SCORE:  0.43501080580426055  acc:  0.4 f1:  0.4098360655737705
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 910.51it/s]


 Average valid loss: 1.19
AUC_SCORE:  0.5530487804878049  acc:  0.4793388429752066 f1:  0.5039370078740157


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 904.98it/s]

 Average valid loss: 1.08
AUC_SCORE:  0.5689024390243902  acc:  0.33884297520661155 f1:  0.5061728395061729
BEST MODEL
AUC_SCORE:  0.5689024390243902  acc:  0.33884297520661155 f1:  0.5061728395061729
{'auc_micro': 0.5689024390243902, 'acc': 0.33884297520661155, 'f1': 0.5061728395061729, 'pred': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1], [], [], [], [], [], [], []]}


# 평균 임베딩

In [19]:
import pickle
with open('emb_avg2.pickle','rb') as fw:
    movie_emb = pickle.load(fw)

In [20]:
path1 = 'word2vec_model_char/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
end = torch.ones(1,1,384)

for filename in filenames:
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'cult film'==keyword:
                    cult = 1
        
    embs = torch.zeros(1,len(movie_emb[movie_name])+1,384)
    for i,emb in enumerate(movie_emb[movie_name]):
        emb = torch.from_numpy(np.nan_to_num(emb))
        embs[0][i] = emb
        
    embs[0][-1] = end
        
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
        #valid_data.append([embs, [cult]])

In [21]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

963
120
121


In [22]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:08<00:00, 118.77it/s]


 Average training loss: 1.11


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 841.80it/s]


 Average valid loss: 1.10
AUC_SCORE:  0.44334671194813213  acc:  0.3416666666666667 f1:  0.5093167701863354
Test


...

 Average valid loss: 1.06
AUC_SCORE:  0.6971287434393331  acc:  0.5166666666666667 f1:  0.546875
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 825.05it/s]


 Average valid loss: 1.09
AUC_SCORE:  0.6820121951219512  acc:  0.47107438016528924 f1:  0.536231884057971


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 821.67it/s]

 Average valid loss: 1.08
AUC_SCORE:  0.6810975609756098  acc:  0.4793388429752066 f1:  0.5401459854014599
BEST MODEL
AUC_SCORE:  0.6810975609756098  acc:  0.4793388429752066 f1:  0.5401459854014599
{'auc_micro': 0.6810975609756098, 'acc': 0.4793388429752066, 'f1': 0.5401459854014599, 'pred': [[1, 1, 0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 0, 1, 1, 1], [1, 1, 1, 1, 0, 1, 1, 1, 1, 1], [1, 1, 0, 0, 1, 1, 1, 1, 1, 1], [1, 0, 1, 0, 1, 1, 1, 1, 0, 1], [1, 1, 1, 0, 0, 1, 1, 0, 0, 1], [1, 1, 0, 1, 1, 0, 1, 1, 1, 1], [1, 0, 1, 0, 1, 0, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 0, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1], [], [], [], [], [], [], []]}


# 그냥 metapath

In [23]:
path1 = 'word2vec_model_char/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'cult film'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [24]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

963
120
121


In [25]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:08<00:00, 120.25it/s]


 Average training loss: 1.10


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 878.15it/s]


 Average valid loss: 1.09
AUC_SCORE:  0.5365853658536586  acc:  0.3416666666666667 f1:  0.5093167701863354
Test


...

 Average valid loss: 0.97
AUC_SCORE:  0.7249150972522383  acc:  0.5833333333333334 f1:  0.5833333333333333
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 870.82it/s]


 Average valid loss: 1.01
AUC_SCORE:  0.7134146341463415  acc:  0.5619834710743802 f1:  0.5891472868217055


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 862.88it/s]


 Average valid loss: 1.02
AUC_SCORE:  0.7140243902439024  acc:  0.5371900826446281 f1:  0.5757575757575758
BEST MODEL
AUC_SCORE:  0.7140243902439024  acc:  0.5371900826446281 f1:  0.5757575757575758
{'auc_micro': 0.7140243902439024, 'acc': 0.5371900826446281, 'f1': 0.5757575757575758, 'pred': [[1, 1, 0, 1, 0, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 0, 0, 1, 1, 1], [1, 1, 1, 1, 0, 0, 1, 1, 1, 1], [1, 1, 0, 0, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 1, 0, 1], [1, 0, 1, 0, 0, 1, 1, 1, 1, 1], [1, 1, 0, 1, 1, 0, 0, 1, 1, 1], [1, 1, 1, 0, 1, 1, 0, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1, 0, 1], [1, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 0, 1, 0, 1, 0, 0], [0], [], [], [], [], [], [], []]}


# Main_act

In [26]:
path1 = 'word2vec_model_extra+emb/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'cult film'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [27]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:08<00:00, 118.98it/s]


 Average training loss: 1.10


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 869.16it/s]

 Average valid loss: 1.08
AUC_SCORE:  0.6545230009262119  acc:  0.3416666666666667 f1:  0.5093167701863354


...

 Average valid loss: 0.99
AUC_SCORE:  0.7332510033961099  acc:  0.5583333333333333 f1:  0.576
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 866.18it/s]


 Average valid loss: 1.05
AUC_SCORE:  0.7054878048780489  acc:  0.5206611570247934 f1:  0.5606060606060607


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 863.78it/s]


 Average valid loss: 0.98
AUC_SCORE:  0.7414634146341463  acc:  0.48760330578512395 f1:  0.5441176470588235
BEST MODEL
AUC_SCORE:  0.7414634146341463  acc:  0.48760330578512395 f1:  0.5441176470588235
{'auc_micro': 0.7414634146341463, 'acc': 0.48760330578512395, 'f1': 0.5441176470588235, 'pred': [[1, 1, 1, 1, 0, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 1, 1, 1], [1, 1, 1, 1, 0, 1, 0, 1, 1, 1], [1, 1, 0, 0, 1, 1, 1, 1, 1, 1], [1, 0, 1, 0, 1, 1, 1, 1, 0, 1], [1, 0, 1, 0, 0, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 0, 1, 1, 1, 1], [1, 1, 1, 0, 1, 1, 0, 1, 0, 1], [0, 1, 1, 1, 1, 1, 1, 1, 0, 1], [1, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 0, 0, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 0, 1, 0, 1], [1], [], [], [], [], [], [], []]}


# pos

In [28]:
path1 = 'word2vec_model_extra+pos/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,400)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'cult film'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,400)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [29]:
bert2 = BertModel(d_model=400).to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=25)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 962/962 [00:08<00:00, 119.27it/s]


 Average training loss: 1.11


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 850.64it/s]


 Average valid loss: 1.08
AUC_SCORE:  0.6523618400740969  acc:  0.3416666666666667 f1:  0.5093167701863354
Test


...

 Average valid loss: 1.00
AUC_SCORE:  0.6995986415560359  acc:  0.525 f1:  0.5365853658536585
Test


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 853.22it/s]


 Average valid loss: 1.01
AUC_SCORE:  0.702994751466502  acc:  0.5583333333333333 f1:  0.5891472868217055


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 854.61it/s]

 Average valid loss: 1.01
AUC_SCORE:  0.704847175054029  acc:  0.5416666666666666 f1:  0.5801526717557252
BEST MODEL
AUC_SCORE:  0.704847175054029  acc:  0.5416666666666666 f1:  0.5801526717557252
{'auc_micro': 0.704847175054029, 'acc': 0.5416666666666666, 'f1': 0.5801526717557252, 'pred': [[1, 1, 0, 0, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 0, 1, 1, 1, 1], [1, 1, 1, 0, 0, 1, 1, 1, 1, 1], [1, 0, 0, 1, 0, 0, 1, 1, 1, 1], [1, 1, 0, 1, 1, 0, 1, 0, 1, 1], [0, 1, 0, 0, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 1, 1, 1, 1, 1], [1, 1, 0, 1, 1, 0, 1, 0, 1, 0], [1, 1, 1, 1, 1, 1, 1, 0, 1, 1], [0, 0, 1, 1, 1, 1, 1, 1, 1, 0], [1, 0, 1, 1, 1, 1, 1, 1, 0, 1], [1, 1, 0, 0, 1, 1, 1, 0, 0, 1], [], [], [], [], [], [], [], []]}


# Text rank

In [30]:
path1 = 'word2vec_model_text/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'cult film'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [31]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 960/960 [00:08<00:00, 119.23it/s]


 Average training loss: 1.11


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 863.19it/s]


 Average valid loss: 1.10
AUC_SCORE:  0.5072553257178142  acc:  0.3416666666666667 f1:  0.5093167701863354
Test


...

 Average valid loss: 1.03
AUC_SCORE:  0.702994751466502  acc:  0.5166666666666667 f1:  0.546875
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 880.17it/s]


 Average valid loss: 1.05
AUC_SCORE:  0.7085365853658536  acc:  0.5289256198347108 f1:  0.5714285714285715


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 874.37it/s]

 Average valid loss: 1.01
AUC_SCORE:  0.7280487804878049  acc:  0.49586776859504134 f1:  0.5547445255474452
BEST MODEL
AUC_SCORE:  0.7280487804878049  acc:  0.49586776859504134 f1:  0.5547445255474452
{'auc_micro': 0.7280487804878049, 'acc': 0.49586776859504134, 'f1': 0.5547445255474452, 'pred': [[1, 1, 0, 1, 0, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 1, 1, 1], [1, 1, 1, 1, 0, 1, 0, 1, 1, 1], [1, 1, 0, 0, 1, 1, 1, 1, 1, 1], [1, 0, 1, 0, 1, 1, 0, 1, 0, 1], [1, 1, 1, 0, 0, 1, 1, 1, 1, 1], [1, 1, 0, 1, 1, 0, 1, 1, 1, 1], [1, 1, 1, 0, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 0, 0, 1, 1, 1, 1, 1, 0], [1, 0, 1, 1, 1, 1, 1, 1, 0, 1], [1], [], [], [], [], [], [], []]}


In [32]:
path1 = 'word2vec_model_text2/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'cult film'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [33]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 960/960 [00:07<00:00, 120.61it/s]


 Average training loss: 1.11


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 869.54it/s]


 Average valid loss: 1.08
AUC_SCORE:  0.6554492127199754  acc:  0.3416666666666667 f1:  0.5093167701863354
Test


...

 Average valid loss: 0.96
AUC_SCORE:  0.7452917567150354  acc:  0.5833333333333334 f1:  0.5833333333333333
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 864.53it/s]


 Average valid loss: 1.09
AUC_SCORE:  0.6810975609756098  acc:  0.5289256198347108 f1:  0.5648854961832062


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 864.32it/s]


 Average valid loss: 1.06
AUC_SCORE:  0.6948170731707317  acc:  0.5206611570247934 f1:  0.5671641791044776
BEST MODEL
AUC_SCORE:  0.6948170731707317  acc:  0.5206611570247934 f1:  0.5671641791044776
{'auc_micro': 0.6948170731707317, 'acc': 0.5206611570247934, 'f1': 0.5671641791044776, 'pred': [[1, 1, 0, 1, 0, 1, 1, 1, 0, 1], [1, 1, 1, 1, 1, 1, 0, 1, 1, 1], [1, 1, 1, 1, 0, 0, 0, 1, 1, 1], [1, 1, 0, 0, 1, 1, 1, 1, 1, 1], [1, 0, 1, 1, 1, 1, 0, 1, 0, 1], [1, 1, 1, 0, 0, 1, 1, 1, 1, 1], [1, 1, 0, 1, 1, 0, 1, 1, 1, 1], [1, 1, 1, 0, 1, 1, 0, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1, 0, 1], [1, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 0, 1, 0, 1, 0, 1], [1], [], [], [], [], [], [], []]}


In [34]:
path1 = 'word2vec_model_text3/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'cult film'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [35]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 960/960 [00:08<00:00, 119.36it/s]


 Average training loss: 1.10


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 707.33it/s]


 Average valid loss: 1.08
AUC_SCORE:  0.693423896264279  acc:  0.3416666666666667 f1:  0.5093167701863354
Test


...

 Average valid loss: 0.99
AUC_SCORE:  0.7434393331275084  acc:  0.575 f1:  0.592
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 863.95it/s]


 Average valid loss: 1.05
AUC_SCORE:  0.6969512195121951  acc:  0.512396694214876 f1:  0.549618320610687


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 866.67it/s]


 Average valid loss: 1.03
AUC_SCORE:  0.7048780487804879  acc:  0.5371900826446281 f1:  0.5625
BEST MODEL
AUC_SCORE:  0.7048780487804879  acc:  0.5371900826446281 f1:  0.5625
{'auc_micro': 0.7048780487804879, 'acc': 0.5371900826446281, 'f1': 0.5625, 'pred': [[1, 1, 0, 1, 0, 1, 1, 1, 0, 1], [1, 1, 1, 1, 1, 0, 0, 1, 1, 1], [1, 1, 1, 1, 0, 1, 1, 1, 1, 0], [1, 1, 0, 0, 1, 1, 1, 1, 1, 1], [1, 1, 1, 0, 0, 1, 0, 1, 0, 1], [1, 0, 1, 0, 0, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 0, 0, 1, 1, 1], [1, 1, 1, 0, 1, 0, 0, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1, 0, 1], [1, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 0, 0, 1, 0, 1, 0, 0], [1], [], [], [], [], [], [], []]}


# 거리로 분석

In [156]:
path1 = 'word2vec_model_dim=384/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'cult film'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1

    ans = torch.zeros(1,1,512)
    for i in range(scene_num):
        if i ==0:
            continue
        #ans[0,0,i] += util.cos_sim(embs[0][i],embs[0][i-1])[0][0]
        ans[0,0,i] += torch.cdist(embs[0][i].unsqueeze(0),embs[0][i-1].unsqueeze(0))[0][0]
        
    embs = ans
    
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [157]:
bert2 = BertModel(d_model=512).to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=20)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 929/929 [00:07<00:00, 127.24it/s]


 Average training loss: 1.07


100%|████████████████████████████████████████| 117/117 [00:00<00:00, 980.24it/s]


 Average valid loss: 1.06
AUC_SCORE:  0.5272873194221509  acc:  0.4700854700854701 f1:  0.3404255319148936
EPOCHS:2
TRAIN


...

 Average training loss: 1.02


100%|████████████████████████████████████████| 117/117 [00:00<00:00, 989.84it/s]


 Average valid loss: 1.13
AUC_SCORE:  0.4562600321027287  acc:  0.47863247863247865 f1:  0.2823529411764706


100%|████████████████████████████████████████| 117/117 [00:00<00:00, 985.66it/s]

 Average valid loss: 1.06
AUC_SCORE:  0.5497592295345104  acc:  0.41025641025641024 f1:  0.3551401869158879
BEST MODEL
AUC_SCORE:  0.5497592295345104  acc:  0.41025641025641024 f1:  0.3551401869158879
{'auc_micro': 0.5497592295345104, 'acc': 0.41025641025641024, 'f1': 0.3551401869158879, 'pred': [[1, 1, 1, 1, 1, 1, 1, 1, 0, 1], [1, 1, 0, 1, 1, 0, 0, 0, 1, 1], [0, 1, 1, 0, 1, 1, 1, 0, 1, 0], [1, 1, 1, 1, 1, 1, 0, 0, 1, 0], [0, 1, 0, 1, 1, 1, 1, 1, 1, 1], [1, 1, 0, 1, 0, 1, 1, 1, 0, 1], [1, 1, 1, 0, 1, 1, 0, 0, 1, 1], [0, 1, 1, 1, 1, 1, 0, 1, 1, 0], [1, 0, 1, 0, 1, 1, 0, 1, 0, 1], [1, 1, 1, 0, 1, 0, 0, 0, 0, 1], [1, 1, 1, 1, 0, 0, 1, 1, 0, 1], [1, 0, 1, 0, 1, 1, 0], [], [], [], [], [], [], []]}
